# Chapter 12

So far, in all the contracts we have written, the flow of the execution was pretty straighforward:
1. The transaction is sent to the contract with a single parameter.
2. The contract uses or not the parameter and runs its code.
3. A pair containing a list of operations and the new storage is returned.

This works well for simple contracts, but what if you want your contract to do multiple things? You could write different separate contracts of course, but sharing their storage and state will be more complicated. In this chapter, you will learn how to change the behaviour of your contract according to the parameters it received!

Let's check a very simple example and see how we can modify it to change its behaviour:

In [4]:
parameter int ;
storage int ;
code {
    UNPAIR ;
    ADD ;
    NIL operation ;
    PAIR ;
} ;

RUN %default 5 6 ;

parameter int;
storage int;
code { { DUP ; CAR ; DIP { CDR } } ; ADD ; NIL operation ; PAIR };
RUN: use %default; drop all; push (5, 6);
DUP: push (5, 6);
CAR: pop (5, 6); push 5;
DIP: protect 1 item(s);
CDR: pop (5, 6); push 6;
restore 1 item(s);
ADD: pop 5, 6; push 11;
NIL: push [];
PAIR: pop [], 11; push ([], 11);

value,type
11,int


This couldn't be more simple 😅 The contract takes the `int` sent in the parameters and adds it to the `int` in the storage.  
Now, imagine you want the same contract to add or subtract two numbers. There must be something in the parameter that tells the contract what to do. This is when `union types` intervene! Let's see first how that would look like:

In [10]:
parameter (or int int) ;
storage int ;
code {
    UNPAIR ;
    IF_LEFT
        { ADD }
        { SUB } ;
    NIL operation ;
    PAIR ;
} ;

RUN %default (Left 5) 6 ;

parameter (or int int);
storage int;
code { { DUP ; CAR ; DIP { CDR } } ; IF_LEFT { ADD } { SUB } ; NIL operation ; PAIR };
RUN: use %default; drop all; push (5, 6);
DUP: push (5, 6);
CAR: pop (5, 6); push 5;
DIP: protect 1 item(s);
CDR: pop (5, 6); push 6;
restore 1 item(s);
IF_LEFT: pop 5; push 5;
ADD: pop 5, 6; push 11;
NIL: push [];
PAIR: pop [], 11; push ([], 11);

value,type
11,int


You can now see that there are three different things in this new contract compared to the previous one:
1. The parameter is `(or (int %increment) (int %decrement))` instead of `int`.
2. There is a conditional structure in the code with **`IF_LEFT`**.
3. The parameters of the **`RUN`** instruction are different.

In order to tell the contract to change its behaviour, we are using a `union type`. As its name indicates, a `union type` is a type made of the union of two other types. In its most basic form, a union type is declared between parentheses with the keyword `or` followed by the two types you want to use. The type on the left is the `left` part of the union, the type on the right the `right` part. After implementing the union type as a parameter, we can pass two different arguments to the contract: a union value with a value on the left or a union value with a value on the right. Now, we want the contract to behave differently according to the value present in the union value. This is what **`IF_LEFT`** does. When we unpair the pair of parameter/storage, the contract knows it expects a union type. If the left side holds a value, the value `(Left value)` will be pushed onto the stack. If the right side holds a value, the value `(Right value)` will be pushed. This is what you can see after the **`RUN`** instruction, we indicated `(Left 5)` as the value we want to use.

As you may have guessed, **`IF_LEFT`** will branch into the first pair of curly braces if the left side of the union value holds a value or it will branch into the second pair of curly braces if the right side of the union value holds a value. After branching, the naked value inside the union value is pushed onto the stack and ready to be used. In the case of this example, the `int` value is added to the value in the storage and the new storage is returned.

To drive the point home, let's see what happens if we want to subtract the value passed as a parameter:

In [11]:
parameter (or int int) ;
storage int ;
code {
    UNPAIR ;
    IF_LEFT
        { ADD }
        { SUB } ;
    NIL operation ;
    PAIR ;
} ;

RUN %default (Right 5) 6 ;

parameter (or int int);
storage int;
code { { DUP ; CAR ; DIP { CDR } } ; IF_LEFT { ADD } { SUB } ; NIL operation ; PAIR };
RUN: use %default; drop all; push (5, 6);
DUP: push (5, 6);
CAR: pop (5, 6); push 5;
DIP: protect 1 item(s);
CDR: pop (5, 6); push 6;
restore 1 item(s);
IF_LEFT: pop 5; push 5;
SUB: pop 5, 6; push -1;
NIL: push [];
PAIR: pop [], -1; push ([], -1);

value,type
-1,int


Pretty simple, we just replace `(Left 5)` with `(Right 5)`! The result of the operation (`-1`) indicates that the contract subtracted the two values instead of adding them together!

Now that you understood the bases of using a union type as a parameter, we can introduce annotations to make the code a little easier to read:

In [12]:
parameter (or (int %increment) (int %decrement)) ;
storage int ;
code {
    UNPAIR ;
    IF_LEFT
        { ADD }
        { SUB } ;
    NIL operation ;
    PAIR ;
} ;

RUN %increment 5 6 ;

parameter (or (int %increment) (int %decrement));
storage int;
code { { DUP ; CAR ; DIP { CDR } } ; IF_LEFT { ADD } { SUB } ; NIL operation ; PAIR };
RUN: use %increment; drop all; push (5, 6);
DUP: push (5, 6);
CAR: pop (5, 6); push 5;
DIP: protect 1 item(s);
CDR: pop (5, 6); push 6;
restore 1 item(s);
IF_LEFT: pop 5; push 5;
ADD: pop 5, 6; push 11;
NIL: push [];
PAIR: pop [], 11; push ([], 11);

value,type
11,int


This contract does the same thing as the one before, the difference is that you can annotate the arguments of the union value and when running the contract, you can use the annotations to target the value of your choice. If you replace `RUN %increment 5 6` with `RUN %decrement 5 6`, you will obtain `-1`. 

Each side of the union type represents the "entrypoint" of the contract and using annotations allows us to identify these entrypoints more clearly. We can say our contract as an "increment" entrypoint and a "decrement" entrypoint.

Now imagine, you want a third entrypoint, for example to reset the storage to zero. Union types only allow two values (left/right). However, nested union types are a thing in Michelson and are widely used in contracts! Let's see how it would look like first without annotations:

In [16]:
parameter (or (or int int) unit) ;
storage int ;
code {
    UNPAIR ;
    IF_LEFT
        {
            IF_LEFT
                { ADD }
                { SUB } ;
        }
        { DROP ; DROP ; PUSH int 0 } ;
    NIL operation ;
    PAIR ;
} ;

RUN %default (Left (Left 5)) 6 ;

parameter (or (or int int) unit);
storage int;
code { { DUP ; CAR ; DIP { CDR } } ; IF_LEFT { IF_LEFT { ADD } { SUB } } { DROP ; DROP ; PUSH int 0 } ; NIL operation ; PAIR };
RUN: use %default; drop all; push (5, 6);
DUP: push (5, 6);
CAR: pop (5, 6); push 5;
DIP: protect 1 item(s);
CDR: pop (5, 6); push 6;
restore 1 item(s);
IF_LEFT: pop 5; push 5;
IF_LEFT: pop 5; push 5;
ADD: pop 5, 6; push 11;
NIL: push [];
PAIR: pop [], 11; push ([], 11);

value,type
11,int


The first **`IF_LEFT`** unwraps the first union value and the second one unwraps the second union value that decides if the contract is going to add the values or subtract them. This contract adds the two values but you can easily subtract by replacing `RUN %default (Left (Left 5)) 6 ;` with `RUN %default (Left (Right 5)) 6 ;`. If you want to reset the storage to `0`, you can write `RUN %default (Right Unit) 6 ;`.

Here is the same contract written with annotations:

In [19]:
parameter (or (or (int %increment) (int %decrement)) (unit %reset)) ;
storage int ;
code {
    UNPAIR ;
    IF_LEFT
        {
            IF_LEFT
                { ADD }
                { SUB } ;
        }
        { DROP ; DROP ; PUSH int 0 } ;
    NIL operation ;
    PAIR ;
} ;

RUN %increment 5 6 ;

parameter (or (or (int %increment) (int %decrement)) (unit %reset));
storage int;
code { { DUP ; CAR ; DIP { CDR } } ; IF_LEFT { IF_LEFT { ADD } { SUB } } { DROP ; DROP ; PUSH int 0 } ; NIL operation ; PAIR };
RUN: use %increment; drop all; push (5, 6);
DUP: push (5, 6);
CAR: pop (5, 6); push 5;
DIP: protect 1 item(s);
CDR: pop (5, 6); push 6;
restore 1 item(s);
IF_LEFT: pop 5; push 5;
IF_LEFT: pop 5; push 5;
ADD: pop 5, 6; push 11;
NIL: push [];
PAIR: pop [], 11; push ([], 11);

value,type
11,int
